In [1]:
from collections import OrderedDict

In [2]:
# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 35)),
    ("jaw", (0, 17))
])

In [3]:
def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
    # create two copies of the image -- one for the
    # overlay and one for the final image output
    overlay = image.copy()
    output = image.copy()
    
    # initialize colors list with a unique color for each facial landmark region
    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
                 (168, 100, 168), (158, 163, 32), (163, 38, 32), (180, 42, 220)]
        
    for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
        # grab the (x, y)-coordinates associated with the face landmark
        (j, k) = FACIAL_LANDMARKS_IDXS[name]
        pts = shape[j:k]
        
        # check if the jawline is to be drawn
        if name == "jaw":
            # since the jawline is a non-enclosed facial region,
            # just draw lines b/w the (x, y)- coordinates
            for l in range(1, len(pts)):
                ptA = tuple(pts[l-1])
                ptB = tuple(pts[l])
                cv2.line(overlay, ptA, ptB, colors[i], 2)
                
        # otherwise, compute the convex hull of the facial
        # landmark coordinates points and display it
        else:
            hull = cv2.convexHull(pts)
            cv2.drawContours(overlay, [hull], -1, colors[i], -1)
            
        # apply the transparent overlay
        cv2.addWeighted(overlay, alpha, output, 1-alpha, 0, output)
        
        return output

In [4]:
from imutils import face_utils
from imutils.video import VideoStream
import datetime
import time
import numpy as np
import argparse
import imutils
import dlib
import cv2

In [5]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-p", "--shape-predictor", required=True,
#     help="path to facial landmark predictor")
# ap.add_argument("-i", "--image", required=True,
#     help="path to input image")
# args = vars(ap.parse_args())

In [6]:
# initialize dlib's face detector and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
#predictor = dlib.shape_predictor(args["shape_predictor"])
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [13]:
# load the input image, resize it, and convert it to grayscale
# image = cv2.imread(args["image"])
image = cv2.imread("images/messi.jpg")
image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale image
rects = detector(gray, 1)

[(247, 26) (290, 70)]


In [8]:
# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a numpy array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    
    #loop over the face parts individually
#     for(name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
#         # clone the original image to draw on it, then
#         # display the name of the face part on the image
#         clone = image.copy()
#         cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
#             0.7, (0, 0, 255), 2)
        
#         #loop over the subset of facial landmarks, drawing the specific face part
#         for (x, y) in shape[i:j]:
#             cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)
        
#         # extract the ROI of the face region as a separate image
#         (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
#         roi = image[y:y+h, x:x+w]
#         roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
        
#         # show the paritcular face part
#         cv2.imshow("ROI", roi)
#         cv2.imshow("Image", clone)
#         cv2.waitKey(0)
        
    # visualize all facial landmarks with a transparent overlay
    output = face_utils.visualize_facial_landmarks(image, shape)
    cv2.startWindowThread()
    cv2.imshow("Image", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
print(shape[27:35])
centre = np.mean(shape[27:35], axis=0)
# print(np.max(shape[27:35], axis=0) - np.min(shape[27:35], axis=0))
nose_width = np.min(np.max(shape[27:35], axis=0) - np.min(shape[27:35], axis=0))
new_nose = cv2.imread("images/nose.png")
new_nose = imutils.resize(new_nose, width = nose_width)
cv2.startWindowThread()
cv2.
cv2.imshow("Nose", new_nose)
cv2.waitKey(0)
cv2.destroyAllWindows()
clone = image.copy()

[[270  39]
 [270  41]
 [270  43]
 [270  46]
 [267  48]
 [268  48]
 [270  49]
 [271  49]]


In [12]:
centre[1]

45.375

In [8]:
vs = VideoStream().start()
time.sleep(2.0)

# loop over the frames in the videostream
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 0)
    
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0,0, 255), -1)
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
    
cv2.destroyAllWindows()
vs.stop()